## Задание 1

In [65]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import scipy as sp

In [66]:
def getGalerkinEqMatrix(n):
    mat = [[0]*n for _ in range(n)]

    for i in range(n):
        m = i + 1
        mat[i][0] = -2/(m+4) + 2/(m+3) + 2/(m+2) - 2/(m+1)
        for j in range(1, n):
            k = j + 1
            mat[i][j] = -2*k/(k+m+3) + (4*k - 2)/(k+m+2) + (k**2 - k + 2)/(k+m+1) - (2 * k **2)/(k+m) + (k * (k-1))/(k+m-1)
    return mat

def getGalerkinRightSide(n):
    res = [0]*n
    for i in range(n):
        m = i + 1
        res[i] = -(1 / (m + 2) - 1 / (m + 3))
    
    return res

def Gauss(A, B):
    n = len(A)
    if n != len(B):
        raise ValueError("A and B must have the same length")
    # Triangle form
    for i in range(n):
        for j in range(i + 1, n):
            coeff = -(A[j][i] / A[i][i])
            for k in range(i, n):
                A[j][k] += coeff * A[i][k]
            B[j] += coeff * B[i]
    # Diagonal form
    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            coeff = -(A[j][i] / A[i][i])
            A[j][i] += coeff * A[i][i]
            B[j] += coeff * B[i]
    # Solution
    res = [B[i] / A[i][i] for i in range(n)]
    return res

def GalerkinSolve(n):
    if n < 1:
        raise ValueError("n must be greater than 0")
    A = getGalerkinEqMatrix(n)
    B = getGalerkinRightSide(n)
    coeffs = Gauss(A, B)

    def func(x):
        return x + sum([coeffs[i - 1] * x**i * (x - 1) for i in range(1, n + 1)])
    
    # for i in range(1, n + 1):
    #     print(f"C{i} = {coeffs[i - 1]}")
    
    return func


In [67]:
def getDifferentiateMatrix(steps):
    if steps < 3:
        raise ValueError("The number of steps must be greater than 3")
    rang = np.linspace(0, 1, steps)
    dist = np.diff(rang)[0]
    n = steps
    mat = [[0]*(n) for _ in range(n)]
    mat[0][0] = 1
    for i in range(n - 2):
        row = i + 1
        mat[row][i] = (1/dist**2 + rang[i]/(dist))
        mat[row][i + 1] = -(2 - 2/dist**2)
        mat[row][i + 2] = (1/dist**2 - rang[i]/(dist))
    mat[-1][-1] = 1/dist
    mat[-1][-2] = -1/dist
    return mat

def getDifferentiateRightSide(steps):
    if steps < 3:
        raise ValueError("The number of steps must be greater than 3")
    rang = np.linspace(0, 1, steps)
    dist = np.diff(rang)[0]
    n = steps
    B = [0] * n
    B[0] = 0
    for i in range(n - 1):
        row = i + 1
        B[row] = -rang[i]
    B[-1] = 1
    return B

def TridiagonalSolve(Mat, F):
    n = len(Mat)
    
    if any(len(row) != n for row in Mat):
        raise ValueError("Матрица должна быть квадратной")
    if len(F) != n:
        raise ValueError("Длина вектора B должна совпадать с размером матрицы")

    A = [0] * n
    B = [0] * n
    C = [0] * n
    
    for i in range(n):
        A[i] = Mat[i][i - 1] if i - 1 >= 0 else 0
        B[i] = Mat[i][i]
        C[i] = Mat[i][i + 1] if i + 1 < n else 0
    
    eps = [0] * (n - 1)
    eta = [0] * (n - 1)
    x = [0] * n
    
    eps[0] = -C[0] / B[0]
    eta[0] = F[0] / B[0]

    # print(f"i={0}, eps={eps[0]}, eta={eta[0]}, Ai={A[0]}, Bi={B[0]}, Ci={C[0]}, F={F[0]}")

    for i in range(1, n - 1):
        denominator = B[i] - A[i] * eps[i - 1]
        eps[i] = C[i] / denominator
        eta[i] = (F[i] + A[i] * eta[i - 1]) / denominator
        # print(f"i={i}, eps={eps[i]}, eta={eta[i]}, Ai={A[i]}, Bi={B[i]}, Ci={C[i]}, F={F[i]}")
    
    # print(f"i={n - 1}, eps={eps[n - 2]}, eta={eta[n - 2]}, Ai={A[n - 1]}, Bi={B[n - 1]}, Ci={C[n - 1]}, F={F[n - 1]}")
            
    x[-1] = (F[-1] - A[-1] * eta[-1]) / (A[-1] * eps[-1] + B[-1])
    for i in range(n - 2, -1, -1):
        x[i] = eta[i] + eps[i] * x[i + 1]
    
    return x
    

In [68]:
Start = 0
End = 1
Steps = 1000


fig = px.line()
xrange = np.linspace(Start, End, Steps)
A = getDifferentiateMatrix(Steps)
B = getDifferentiateRightSide(Steps)
diffSolution = TridiagonalSolve(A, B)
# diffSolution = Gauss(A, B)
print(diffSolution[-1])
for i in [3, 5, 6, 7, 8]:
    sol = GalerkinSolve(i)
    fig.add_scatter(x=xrange, y=sol(xrange), name=f"n = {i}")
fig.add_scatter(x=xrange, y=diffSolution, name="Differentiation")
fig.show()


0.4877628541467046


# Задание 2

## Из лабы 2


In [69]:
from decimal import Decimal, getcontext

c = Decimal('3') * Decimal('1e10')
R = Decimal('0.35')

def T_z(z):
    z = Decimal(str(z))
    Tw = Decimal('2000')
    To = Decimal('1e4')
    p = Decimal('4')
    return (Tw - To) * (z ** p) + To

def up_z(z):
    z = Decimal(str(z))
    const1 = Decimal('3.084E-4')
    const2 = Decimal('4.799E4')
    T = T_z(z)
    denominator = (const2 / T).exp() - Decimal('1')
    return const1 / denominator


def read_data_kT(variant=2):
    data_tab = []
    with open('data/kT.txt') as file:
        for line in file:
            if line.strip():
                parts = line.split()
                T = Decimal(parts[1])
                k_val = Decimal(parts[2] if variant == 1 else parts[3]) # / Decimal(2)
                data_tab.append([T, k_val])
    return data_tab

def linear_interpolation(x, x_values, y_values):
    for i in range(len(x_values)-1):
        if x_values[i] <= x <= x_values[i+1]:
            return y_values[i] + (y_values[i+1] - y_values[i]) * (x - x_values[i]) / (x_values[i+1] - x_values[i])
    # Экстраполяция если x за границами
    if x < x_values[0]:
        return y_values[0]
    return y_values[-1]

def getk_zFunc(variantkT):
    data_kT = read_data_kT(variantkT)
    log_T = [d[0].ln() for d in data_kT]
    log_k = [d[1].ln() for d in data_kT]
    
    def k_z(z):
        T_dec = Decimal(T_z(z))
        x = T_dec.ln()
        interp_log_k = linear_interpolation(x, log_T, log_k)
        return interp_log_k.exp()
    
    return k_z
# Установка точности вычислений
getcontext().prec = 50
EPS = Decimal('1e-37')  # Decimal('1e-37')

z0 = Decimal('0')
F0 = Decimal('0')
zTarget = Decimal('1')
def get_y0(khi):
    khi = Decimal(str(khi))
    return khi * up_z(z0)

def du(zi, ui, Fi, k_z):
    zi = Decimal(str(zi))
    ui = Decimal(str(ui))
    Fi = Decimal(str(Fi))
    k_val = k_z(zi)
    # return -Decimal('3') * k_val * Fi / c
    return - Fi * R * k_val

def dF(zi, ui, Fi, k_z):
    zi = Decimal(str(zi))
    ui = Decimal(str(ui))
    Fi = Decimal(str(Fi))
    k_val = k_z(zi)
    up = up_z(zi)
    
    if zi == Decimal('0') and Fi == Decimal('0'):
        return Decimal('0.5') * Decimal('3') * R * k_val * (up - ui)
    return -Fi / zi + Decimal('3') * R * k_val * (up - ui)

# Методы Рунге-Кутта
def RungeKutta4Step(duFunc, dFFunc, zi, ui, Fi, h, k_z):
    h = Decimal(str(h))
    zi = Decimal(str(zi))
    ui = Decimal(str(ui))
    Fi = Decimal(str(Fi))
    
    k1 = duFunc(zi, ui, Fi, k_z)
    q1 = dFFunc(zi, ui, Fi, k_z)
    
    h2 = h / Decimal('2')
    k2 = duFunc(zi + h2, ui + h2 * k1, Fi + h2 * q1, k_z)
    q2 = dFFunc(zi + h2, ui + h2 * k1, Fi + h2 * q1, k_z)
    
    k3 = duFunc(zi + h2, ui + h2 * k2, Fi + h2 * q2, k_z)
    q3 = dFFunc(zi + h2, ui + h2 * k2, Fi + h2 * q2, k_z)
    
    k4 = duFunc(zi + h, ui + h * k3, Fi + h * q3, k_z)
    q4 = dFFunc(zi + h, ui + h * k3, Fi + h * q3, k_z)
    
    yip1 = ui + (h/Decimal('6')) * (k1 + Decimal('2')*k2 + Decimal('2')*k3 + k4)
    zip1 = Fi + (h/Decimal('6')) * (q1 + Decimal('2')*q2 + Decimal('2')*q3 + q4)
    
    # if yip1 < 0:
    #     print(f"RK4step: yip1={float(yip1)}, k1={float(k1)} k2={float(k2)} k3={float(k3)} k4={float(k4)}, ri={float(ri)}, ui={float(ui)}, Fi={float(Fi)}")
    
    return yip1, zip1

def RungeKutta4(duFunc, dFFunc, z0, y0, F0, h, zTarget, k_z):
    z0 = Decimal(str(z0))
    y0 = Decimal(str(y0))
    z0 = Decimal(str(z0))
    h = Decimal(str(h))
    zTarget = Decimal(str(zTarget))
    
    zvals = [z0]
    yvals = [y0]
    Fvals = [F0]
    
    while zvals[-1] < zTarget:
        ri = zvals[-1] + h
        yi, Fi = RungeKutta4Step(duFunc, dFFunc, zvals[-1], yvals[-1], Fvals[-1], h, k_z)
        zvals.append(ri)
        yvals.append(yi)
        Fvals.append(Fi)
    
    return zvals, yvals, Fvals

# Метод стрельбы и половинного деления
def solutionShooting(khi, RungeKuttaFunc, h_start, k_z):
    khi = Decimal(str(khi))
    h = Decimal(str(h_start))
    y0 = get_y0(khi)
    
    res_h = RungeKuttaFunc(du, dF, z0, y0, F0, h, zTarget, k_z)
    return (res_h[1][-1], res_h[2][-1]), h

def equationTarget(uR, FR):
    uR = Decimal(str(uR))
    FR = Decimal(str(FR))
    return FR - Decimal('3') * Decimal('0.39') * uR

def sign(val):
    if abs(val) < EPS:
        return 0
    elif val < 0:
        return -1
    return 1

def HalfDivMethod(RungeKuttaFunc, h_start, k_z):
    h = Decimal(str(h_start))
    khi_l, khi_r = Decimal('0'), Decimal('1')
    
    res_l, h = solutionShooting(khi_l, RungeKuttaFunc, h, k_z)
    res_r, h = solutionShooting(khi_r, RungeKuttaFunc, h, k_z)
    sign_l, sign_r = sign(equationTarget(*res_l)), sign(equationTarget(*res_r))

    while abs((khi_r - khi_l) / khi_r) >= EPS:    #  sign_l != sign_r:
        khi_mid = (khi_l + khi_r) / Decimal('2')
        res_mid, h = solutionShooting(khi_mid, RungeKuttaFunc, h, k_z)
        print(f"khim = {khi_mid}, abs={abs((khi_r - khi_l) / khi_r)}, sign_l={sign_l}, khi_r= {khi_r}, yR = {res_mid[0]}, FR = {res_mid[1]} h = {h}")
        # sign_mid = sign(equationTarget(*res_mid))
        sign_mid = sign(equationTarget(*res_mid) * equationTarget(*res_l))

        if sign_mid == 0:
            return khi_mid, h
        elif sign_mid == sign_l:
            khi_l = khi_mid
            res_l = res_mid[:]
        else:
            khi_r = khi_mid
            res_r = res_mid[:]
    
    return khi_mid, h

# # Запуск расчета
# k_z = getk_zFunc(variantkT=2)
# h_st = Decimal('0.001')
# khi_solution, h_solution = HalfDivMethod(RungeKutta4, h_st, k_z)
# print(f"Решение: khi = {khi_solution}, h = {h_solution}")

# var 2
khi_solution = Decimal('0.9999988534766292698348550258036927915402001495983')
h_solution = Decimal('0.001')
k_z = getk_zFunc(variantkT=2)

# var 1
# khi_solution = Decimal('0.1537611456568583889747969806194305419922') 
# h_solution = Decimal('0.001')
# k_z = getk_zFunc(variantkT=1)

In [70]:
from decimal import Decimal
import plotly.express as px
import pandas as pd
from IPython.display import display


# Выбор метода и начальных условий
RungeKutta = RungeKutta4
y0 = get_y0(khi_solution)
print(f"khi = {khi_solution}, h = {h_solution}")

# Вычисление решений (все вычисления остаются в Decimal)
zvals, uvals, Fvals = RungeKutta(du, dF, z0, y0, F0, h_solution, zTarget, k_z)
upvals = [up_z(zi) for zi in zvals]

# Функция преобразования Decimal для визуализации
def decimal_to_float(x):
    return float(x) if isinstance(x, Decimal) else x

# Создание DataFrame
def create_decimal_df(zvals, yvals, label):
    return pd.DataFrame({
        "z": [decimal_to_float(z) for z in zvals],
        "f": [decimal_to_float(y) for y in yvals], 
        "label": [label] * len(zvals)
    })

print(len(zvals))

zvals = [decimal_to_float(z) for z in zvals]
uvals = [decimal_to_float(u) for u in uvals]
Fvals = [decimal_to_float(F) for F in Fvals]



fig = px.line()
fig.add_scatter(x=zvals, y=uvals, name="u(z)")
fig.add_scatter(x=zvals, y=upvals, name="u_p(z)")
fig.show()


fig = px.line()
fig.add_scatter(x=zvals, y=Fvals, name="F(z)")
fig.show()

Task2Steps = len(zvals)

SolutionTask2 = uvals.copy()
print(SolutionTask2)
FTask2 = Fvals.copy()


khi = 0.9999988534766292698348550258036927915402001495983, h = 0.001
1001


[2.5616935833510398e-06, 2.561693572544179e-06, 2.5616935400141508e-06, 2.561693485401211e-06, 2.56169340811019e-06, 2.5616933073080506e-06, 2.561693181923781e-06, 2.561693030648377e-06, 2.5616928519348368e-06, 2.56169264399816e-06, 2.5616924048153467e-06, 2.561692132125396e-06, 2.5616918234293073e-06, 2.5616914759900795e-06, 2.561691086832711e-06, 2.5616906527441984e-06, 2.5616901702735392e-06, 2.561689635731728e-06, 2.5616890451917593e-06, 2.5616883944886266e-06, 2.561687679219322e-06, 2.5616868947428382e-06, 2.5616860361801653e-06, 2.561685098414294e-06, 2.5616840760902147e-06, 2.5616829636149185e-06, 2.561681755157397e-06, 2.561680444648643e-06, 2.561679025781653e-06, 2.5616774920114244e-06, 2.561675836554961e-06, 2.5616740523912706e-06, 2.5616721322613686e-06, 2.5616700686682783e-06, 2.561667853877033e-06, 2.561665479914678e-06, 2.5616629385702742e-06, 2.5616602213948973e-06, 2.5616573197016447e-06, 2.5616542245656337e-06, 2.5616509268240098e-06, 2.561647417075947e-06, 2.561643685

## Решение лабы 3

In [71]:
Tw = 2000
T0 = 10000
R = 0.35
p = 4

c = 3 * 1e10


# TempTable = ((2000, 8.200E-03),
#     (3000, 2.768E-02),
#     (4000, 6.560E-02),
#     (5000, 1.281E-01),
#     (6000, 2.214E-01),
#     (7000, 3.516E-01),
#     (8000, 5.248E-01),
#     (9000, 7.472E-01),
#     (10000, 1.025E+00),
# )

TempTable = ((2000, 1.6),
    (3000, 5.4),
    (4000, 1.280E+01),
    (5000, 2.500E+01),
    (6000, 4.320E+01),
    (7000, 6.860E+01),
    (8000, 1.024E+02),
    (9000, 1.458E+02),
    (10000, 2.000E+02),
)

def getkFunc():
    tarr, karr = zip(*TempTable)
    tarr = [np.log(x) for x in tarr]
    karr = [np.log(x) for x in karr]
    def k(t):
        return np.exp(np.interp([np.log(t)], tarr, karr)[0])
    return k



def T(z):
    return (Tw - T0) * (z)**p + T0

def Plank(z):
    return 3.084 * 1e-4 / (np.exp(4.799*1e4 / T(z)) - 1)

def formTridiagonalForm(n):
    z = np.linspace(0, 1, n)
    h = np.diff(z)[0]
    tk = getkFunc()
    k = lambda x: tk(T(x))

    mat = [[0] * n for _ in range(n)]
    
    kappa = lambda x: 2 / (k(x) + k(x + h))
    mat[0][0] =  1/((k(z[0]) + k(z[1])) * R)
    mat[0][1] = -mat[0][0]
    for i in range(1, n - 1):
        A = -(z[i] - h/2)*kappa(z[i - 1]) / R**2 / h
        C = -(z[i] + h/2)*kappa(z[i]) / R**2 / h
        B = (A + C - 3*k(z[i])*z[i]*h)
        mat[i][i - 1] = A
        mat[i][i] = B
        mat[i][i + 1] = C
    mat[-1][-2] = - (1/(k(z[-2]) + k(z[-1]))/R**2/h*(2 - h))
    # mat[-1][-2] = -kappa(z[-2]) * (1 - h/2) /  R**2 / h # MN
    # mat[-1][-1] = 3*0.39 - mat[-1][-2] + 3*R*k(z[-1]) * h/2
    mat[-1][-1] = 3*0.39 / R - mat[-1][-2]
    
    return mat

def formTridiagonalSide(n):
    z = np.linspace(0, 1, n)
    h = np.diff(z)[0]
    tk = getkFunc()
    k = lambda x: tk(T(x))

    arr = [0] * n
    for i in range(1, n - 1):
        arr[i] = -3 * k(z[i]) * z[i] * h * Plank(z[i])
    # arr[-1] = 3 * R * k(z[-1]) * z[-1] * h/2 * Plank(z[-1])
    return arr

def IntegrF(z, u : list[float]):
    k = getkFunc()
    F = [0] * len(u)
    intFunc = [0]
    for i in range(1, len(u)):
        intFunc.append(z[i] * k(T(z[i])) * (Plank(z[i]) - u[i]))
        F[i] = 3*R/z[i] * sp.integrate.simpson(intFunc, z[:i+1])
    return F

def DivF(z, u, F):
    k = getkFunc()
    return 3 * R * k(T(z)) * (Plank(z) - u) - F / z

def ApproxF(z, u : list[float]):
    h = np.diff(z)[0]
    k = getkFunc()
    F = [0] * len(u)
    for i in range(1, len(u) - 1):
        F[i] = - 1 / R / k(T(z[i])) * (u[i + 1] - u[i - 1]) / 2 / h
    F[-1] = - 1 / R / k(T(z[-1])) * (u[-1] * (1 - 3 * 0.39 * R * k(T(z[-1])) * h) - u[-2]) / 2 / h
    return F

        
Start = 0
End = 1
Steps = 10

A = formTridiagonalForm(Steps)
B = formTridiagonalSide(Steps)
sol = TridiagonalSolve(A, B)
xrange = np.linspace(Start, End, Steps)
Fint = IntegrF(xrange, sol)
Fapprox = ApproxF(xrange, sol)

fig = px.line()
fig.add_scatter(x=xrange, y=Plank(xrange), name="u_p(z)")
fig.add_scatter(x=xrange, y=sol, name="u(z)")
fig.add_scatter(x=xrange, y=Fint, name="Fint(z)")
fig.add_scatter(x=xrange, y=Fapprox, name="Fapprox(z)")
fig.add_scatter(x=xrange, y=DivF(xrange, sol, Fint), name="DivF(z)")
fig.show()

MaxFDif = abs(Fint[0] - Fapprox[0])
MinFDif = abs(Fint[0] - Fapprox[0])
SumFDif = abs(Fint[0] - Fapprox[0]) 
for i in range(1, len(Fint)):
    MinFDif = min(MinFDif, abs(Fint[i] - Fapprox[i]) / (Fint[i]))
    SumFDif += abs(Fint[i] - Fapprox[i]) / (Fint[i])
    MaxFDif = max(MaxFDif, abs(Fint[i] - Fapprox[i]) / (Fint[i]))

print("MaxFDif = ", MaxFDif)
print("MinFDif = ", MinFDif)
print("AvgFDif = ", SumFDif / len(Fint))



Task3Steps = Steps
SolutionTask3 = sol
FintTask3 = Fint
FapproxTask3 = Fapprox
print("yN = ", sol[-1])
print("FN=", Fint[-1])

/tmp/ipykernel_4120/140813181.py:95: RuntimeWarning:

invalid value encountered in divide



MaxFDif =  6.201453322420827
MinFDif =  0
AvgFDif =  0.7294655937198742
yN =  3.4184127509431146e-08
FN= 2.729551411451593e-08


In [72]:
xrange2 = np.linspace(Start, End, Task2Steps)
xrange3 = np.linspace(Start, End, Task3Steps)

MinFDif = abs(FintTask3[0] - FTask2[0])
MaxFDif = abs(FintTask3[0] - FTask2[0])
SumFDif = abs(FintTask3[0] - FTask2[0]) 

for i in range(1, len(xrange3)):
    MinFDif = min(MinFDif, abs(FintTask3[i] - FTask2[i]) / abs(FTask2[i]))
    MaxFDif = max(MaxFDif, abs(FintTask3[i] - FTask2[i]) / abs(FTask2[i]))
    SumFDif = SumFDif + abs(FintTask3[i] - FTask2[i]) / abs(FTask2[i])

print("MinFDif: ", MinFDif)
print("MaxFDif: ", MaxFDif)
print("AvgFDif: ", SumFDif / len(xrange3))

MinUDif = abs(SolutionTask3[0] - SolutionTask2[0]) / abs(SolutionTask2[0])
MaxUDif = abs(SolutionTask3[0] - SolutionTask2[0]) / abs(SolutionTask2[0])
SumUDif = abs(SolutionTask3[0] - SolutionTask2[0]) / abs(SolutionTask2[0])

for i in range(1, len(xrange3)):
    MinUDif = min(MinUDif, abs(SolutionTask3[i] - SolutionTask2[i]) / abs(SolutionTask2[i]))
    MaxUDif = max(MaxUDif, abs(SolutionTask3[i] - SolutionTask2[i]) / abs(SolutionTask2[i]))
    SumUDif = SumUDif + abs(SolutionTask3[i] - SolutionTask2[i]) / abs(SolutionTask2[i])

print("MinUDif: ", MinUDif)
print("MaxUDif: ", MaxUDif)
print("AvgUDif: ", SumUDif / len(xrange3))




fig = px.line()
fig.add_scatter(x=xrange3, y=Plank(xrange3), name="u_p(z)")
fig.add_scatter(x=xrange2, y=SolutionTask2, name="Task 2 u(z)")
fig.add_scatter(x=xrange3, y=SolutionTask3, name="Task 3 u(z)")
fig.show()

fig = px.line()
fig.add_scatter(x=xrange2, y=FTask2, name="Task 2 F(z)")
fig.add_scatter(x=xrange3, y=FintTask3, name="Task 3 F(z) integral")
fig.add_scatter(x=xrange3, y=FapproxTask3, name="Task 3 F(z) approximation")
fig.show()

MinFDif:  0.0
MaxFDif:  79836.29737656207
AvgFDif:  34692.65428204737
MinUDif:  0.0006632400165966503
MaxUDif:  0.9866556483309885
AvgUDif:  0.39605935358783684
